In [1]:
import pandas as pd
df_raw = pd.read_csv('data/raw_data.csv')
df_incremental = pd.read_csv('data/incremental_data.csv')

In [2]:
#TRANSFORMATIONS
#Handling missing values in raw and incremental data
# Fill missing values with "Unknown" for specific columns in both datasets
print("Missing values before (raw):\n", df_raw.isnull().sum())
print("Missing values before (incremental):\n", df_incremental.isnull().sum())

df_raw.fillna({"region": "Unknown", "customer_name": "Unknown","quantity":"Unknown","unit_price":"Unknown","order-data":"Unknown","region":"Unknown"}, inplace=True)
df_incremental.fillna({"region": "Unknown", "customer_name": "Unknown","quantity":"Unknown","region":"Unknown"}, inplace=True)

print("\nMissing values after (raw):\n", df_raw.isnull().sum())
print("Missing values after (incremental):\n", df_incremental.isnull().sum())

#dropping duplicates in raw and incremental data
print("\nDuplicates before (raw):", df_raw.duplicated().sum())
print("Duplicates before (incremental):", df_incremental.duplicated().sum())
df_raw.drop_duplicates(inplace=True)
df_incremental.drop_duplicates(inplace=True)
print("Duplicates after (raw):", df_raw.duplicated().sum())
print("Duplicates after (incremental):", df_incremental.duplicated().sum())



Missing values before (raw):
 order_id          0
customer_name     1
product           0
quantity         26
unit_price       35
order_date        1
region           25
dtype: int64
Missing values before (incremental):
 order_id         0
customer_name    6
product          0
quantity         4
unit_price       0
order_date       0
region           2
dtype: int64

Missing values after (raw):
 order_id         0
customer_name    0
product          0
quantity         0
unit_price       0
order_date       1
region           0
dtype: int64
Missing values after (incremental):
 order_id         0
customer_name    0
product          0
quantity         0
unit_price       0
order_date       0
region           0
dtype: int64

Duplicates before (raw): 1
Duplicates before (incremental): 0
Duplicates after (raw): 0
Duplicates after (incremental): 0


In [3]:
#converting dates and changing data types
#converting order_date column in both datasets to datetime format
#ensures that the date is in a consistent format for analysis
# Simulate order_date as string/object
df_raw["order_date"] = df_raw["order_date"].astype(str)
df_incremental["order_date"] = df_incremental["order_date"].astype(str)

# Before
print("Date type before:", df_raw["order_date"].dtype)
print("Date type before:", df_incremental["order_date"].dtype)

# Convert to datetime
df_raw["order_date"] = pd.to_datetime(df_raw["order_date"], errors="coerce")
df_incremental["order_date"] = pd.to_datetime(df_incremental["order_date"], errors="coerce")

# After
print("Date type after:", df_raw["order_date"].dtype)
print("Date type after:", df_incremental["order_date"].dtype)

Date type before: object
Date type before: object
Date type after: datetime64[ns]
Date type after: datetime64[ns]


In [5]:
#Dropping unnecessary columns in raw and incremental data
#Removing rows where quantity or unit_price <= 0
#Rows with quantity or unit_price <= 0 will not be useful for analysis, so we will drop them.

#Drop rows where quantity or unit_price <= 0

df_raw['quantity'] = pd.to_numeric(df_raw['quantity'], errors='coerce')
df_raw['unit_price'] = pd.to_numeric(df_raw['unit_price'], errors='coerce')
df_incremental['quantity'] = pd.to_numeric(df_incremental['quantity'], errors='coerce')
df_incremental['unit_price'] = pd.to_numeric(df_incremental['unit_price'], errors='coerce')

print("Rows before filtering:", df_raw.shape[0])
df_raw = df_raw[(df_raw['quantity'] > 0) & (df_raw['unit_price'] > 0)]
print("Rows after filtering:", df_raw.shape[0])

print("Rows before filtering:", df_incremental.shape[0])
df_incremental = df_incremental[(df_incremental['quantity'] > 0) & (df_incremental['unit_price'] > 0)]
print("Rows after filtering:", df_incremental.shape[0])

# Dropping unnecessary columns in incremental data
print("Columns before dropping (incremental):", df_incremental.columns)
cols_to_drop_inc = [col for col in ['notes', 'extra_info'] if col in df_incremental.columns]
df_incremental.drop(columns=cols_to_drop_inc, inplace=True)
print("Columns after dropping (incremental):", df_incremental.columns)

# Dropping unnecessary columns in raw data
print("Columns before dropping (raw):", df_raw.columns)
cols_to_drop_inc = [col for col in ['notes', 'extra_info'] if col in df_raw.columns]
df_incremental.drop(columns=cols_to_drop_inc, inplace=True)
print("Columns after dropping (raw):", df_raw.columns)



Rows before filtering: 99
Rows after filtering: 46
Rows before filtering: 10
Rows after filtering: 6
Columns before dropping (incremental): Index(['order_id', 'customer_name', 'product', 'quantity', 'unit_price',
       'order_date', 'region'],
      dtype='object')
Columns after dropping (incremental): Index(['order_id', 'customer_name', 'product', 'quantity', 'unit_price',
       'order_date', 'region'],
      dtype='object')
Columns before dropping (raw): Index(['order_id', 'customer_name', 'product', 'quantity', 'unit_price',
       'order_date', 'region'],
      dtype='object')
Columns after dropping (raw): Index(['order_id', 'customer_name', 'product', 'quantity', 'unit_price',
       'order_date', 'region'],
      dtype='object')


In [6]:
#Categorization
#create a new column customer_tier that categorizes customers by their total purchase amount
#This will help in understanding customer value and targeting marketing efforts.
# Add total_price column if not already present
df_raw["total_price"] = df_raw["quantity"] * df_raw["unit_price"]
df_incremental["total_price"] = df_incremental["quantity"] * df_incremental["unit_price"]

# Before categorization
print("Customer tier column before:", "customer_tier" in df_raw.columns)
print("Customer tier column before (incremental):", "customer_tier" in df_incremental.columns)

# Categorize customers by total_price
bins = [0, 100, 500, 1000, float('inf')]
labels = ["Bronze", "Silver", "Gold", "Platinum"]
df_raw["customer_tier"] = pd.cut(df_raw["total_price"], bins=bins, labels=labels)
df_incremental["customer_tier"] = pd.cut(df_incremental["total_price"], bins=bins, labels=labels)

# After categorization
print("Customer tier column after:", "customer_tier" in df_raw.columns)
print("Unique tiers:", df_raw["customer_tier"].unique())

Customer tier column before: False
Customer tier column before (incremental): False
Customer tier column after: True
Unique tiers: ['Silver', 'Platinum', 'Gold']
Categories (4, object): ['Bronze' < 'Silver' < 'Gold' < 'Platinum']


In [7]:
#save the transformed dataframes
df_raw.to_csv('transformed/transformed_full.csv', index=False)
df_incremental.to_csv('transformed/transformed_incremental.csv', index=False)

print("Transformed files saved to 'transformed/' folder.")


Transformed files saved to 'transformed/' folder.
